In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler #you can use minmax scaler too
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
#Import other necessary model libraries, for this example, using Logistic Regression

In [3]:
train_data = pd.read_csv(r"../Data/Corrected Datasets/train-dataset.csv", header = 0)
test_data = pd.read_csv(r"../Data/Corrected Datasets/test-dataset.csv", header = 0)
validation_data = pd.read_csv(r"../Data/Corrected Datasets/validation-dataset.csv", header = 0)

In [4]:
data_quan_cols = ['Age','Discount_Rate', 'Room_Rate','Expected_stay_days', 
                        'Reservation_gap', 'Adults', 'Children','Babies']

In [5]:
data_cat_cols = ['Gender', 'Ethnicity', 'Educational_Level', 'Income', 'Country_region',
                 'Hotel_Type', 'Meal_Type', 'Visted_Previously',
                 'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
                 'Required_Car_Parking', 'Use_Promotion']

In [6]:
quan_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

quan_transformed = quan_pipeline.fit_transform(train_data[data_quan_cols])

In [7]:
data_pipeline = ColumnTransformer([
    ('numerical', quan_pipeline, data_quan_cols),
    ('categorical', OneHotEncoder(), data_cat_cols),
    
])

train_data_processed = data_pipeline.fit_transform(train_data)

In [8]:
validation_data_processed = data_pipeline.transform(validation_data)

In [9]:
type(train_data_processed)

numpy.ndarray

In [10]:
le = preprocessing.LabelEncoder()

In [11]:
target_train = train_data['Reservation_Status']
le.fit(target_train)

LabelEncoder()

In [12]:
target_validation = validation_data['Reservation_Status']

In [13]:
print(le.classes_)

['Canceled' 'Check-In' 'No-Show']


In [14]:
y_train = le.transform(target_train)
y_valid = le.transform(target_validation)

In [15]:
y_valid

array([2, 0, 0, ..., 2, 1, 1])

In [80]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators= 98, criterion= 'entropy')

In [81]:
model.fit(train_data_processed, target_train)

BalancedBaggingClassifier(base_estimator=RandomForestClassifier(class_weight='balanced',
                                                                max_depth=5,
                                                                n_estimators=300),
                          random_state=0)

In [82]:
y_pred = model.predict(validation_data_processed)

In [83]:
print(y_pred)

['Canceled' 'Canceled' 'No-Show' ... 'Canceled' 'No-Show' 'No-Show']


In [85]:
# Printing confusion matrix
confusion_matrix(y_valid, y_pred)

array([[199, 227, 315],
       [414, 571, 625],
       [108, 128, 162]])

In [86]:
# Getting f1 score
from sklearn.metrics import classification_report
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.28      0.27      0.27       741
           1       0.62      0.35      0.45      1610
           2       0.15      0.41      0.22       398

    accuracy                           0.34      2749
   macro avg       0.35      0.34      0.31      2749
weighted avg       0.46      0.34      0.37      2749



In [38]:
ran_for_params = {
    'n_estimators' : range(1,100),
    'criterion': ['gini', 'entropy'],
    #'max_depth': range(1,10),
#     'min_samples_leaf': range(1,10),
#     'min_samples_split': range(1,10),
    #'bootstrap' : [True, False]
}

In [40]:
random_forest_model = RandomForestClassifier()
model_rand_fors = GridSearchCV(random_forest_model, ran_for_params, cv=5, n_jobs = -1, verbose = 1)
model_rand_fors.fit(train_data_processed, target_train)

Fitting 5 folds for each of 198 candidates, totalling 990 fits


ValueError: Invalid parameter criterion for estimator BalancedBaggingClassifier(base_estimator=RandomForestClassifier(),
                          random_state=0). Check the list of available parameters with `estimator.get_params().keys()`.

In [35]:
model_rand_fors.best_params_

{'criterion': 'entropy', 'n_estimators': 98}

In [36]:
model_rand_fors.best_score_

0.7684275091338921